# Open Vaccine - projet Kaggle

## Environnement

In [47]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from collections import Counter as count
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import KFold

## Les données 

In [3]:
train = pd.read_json('train.json', lines=True)
test = pd.read_json('test.json', lines=True)
sub = pd.read_csv('sample_submission.csv')

print('Train dimensions: ', train.shape)
print('Test dimensions: ', test.shape)

Train dimensions:  (2400, 19)
Test dimensions:  (3634, 7)


In [5]:
train.head(5)

,index,id,sequence,structure,predicted_loop_type,signal_to_noise,SN_filter,seq_length,seq_scored,reactivity_error,deg_error_Mg_pH10,deg_error_pH10,deg_error_Mg_50C,deg_error_50C,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUA...,.....((((((.......)))).)).((.....((..((((((......,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHH...,6.894,1,107,68,"[0.1359, 0.20700000000000002, 0.1633, 0.1452, ...","[0.26130000000000003, 0.38420000000000004, 0.1...","[0.2631, 0.28600000000000003, 0.0964, 0.1574, ...","[0.1501, 0.275, 0.0947, 0.18660000000000002, 0...","[0.2167, 0.34750000000000003, 0.188, 0.2124, 0...","[0.3297, 1.5693000000000001, 1.1227, 0.8686, 0...","[0.7556, 2.983, 0.2526, 1.3789, 0.637600000000...","[2.3375, 3.5060000000000002, 0.3008, 1.0108, 0...","[0.35810000000000003, 2.9683, 0.2589, 1.4552, ...","[0.6382, 3.4773, 0.9988, 1.3228, 0.78770000000..."
1,1,id_0049f53ba,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACC...,.....(((((((((((((((((((((((....)))))))))).)))...,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSS...,0.193,0,107,68,"[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.567...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[10.1986, 9.2418, 5.0933, 5.0933, 5.0933, 5.09...","[16.6174, 13.868, 8.1968, 8.1968, 8.1968, 8.19...","[15.4857, 7.9596, 13.3957, 5.8777, 5.8777, 5.8...","[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.947, 4.4523, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[4.8511, 4.0426, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[7.6692, 0.0, 10.9561, 0.0, 0.0, 0.0, 0.0, 0.0..."
2,2,id_006f36f57,GGAAAGUGCUCAGAUAAGCUAAGCUCGAAUAGCAAUCGAAUAGAAU...,.....((((.((.....((((.(((.....)))..((((......)...,EEEEESSSSISSIIIIISSSSMSSSHHHHHSSSMMSSSSHHHHHHS...,8.800,1,107,68,"[0.0931, 0.13290000000000002, 0.11280000000000...","[0.1365, 0.2237, 0.1812, 0.1333, 0.1148, 0.160...","[0.17020000000000002, 0.178, 0.111, 0.091, 0.0...","[0.1033, 0.1464, 0.1126, 0.09620000000000001, ...","[0.14980000000000002, 0.1761, 0.1517, 0.116700...","[0.44820000000000004, 1.4822, 1.1819, 0.743400...","[0.2504, 1.4021, 0.9804, 0.49670000000000003, ...","[2.243, 2.9361, 1.0553, 0.721, 0.6396000000000...","[0.5163, 1.6823000000000001, 1.0426, 0.7902, 0...","[0.9501000000000001, 1.7974999999999999, 1.499..."
3,3,id_0082d463b,GGAAAAGCGCGCGCGCGCGCGCGAAAAAGCGCGCGCGCGCGCGCGC...,......((((((((((((((((......))))))))))))))))((...,EEEEEESSSSSSSSSSSSSSSSHHHHHHSSSSSSSSSSSSSSSSSS...,0.104,0,107,68,"[3.5229, 6.0748, 3.0374, 3.0374, 3.0374, 3.037...","[73705.3985, 73705.3985, 73705.3985, 73705.398...","[11.8007, 12.7566, 5.7733, 5.7733, 5.7733, 5.7...","[121286.7181, 121286.7182, 121286.7181, 121286...","[15.3995, 8.1124, 7.7824, 7.7824, 7.7824, 7.78...","[0.0, 2.2399, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[0.0, -0.5083, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[3.4248, 6.8128, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, -0.8365, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[7.6692, -1.3223, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,4,id_0087940f4,GGAAAAUAUAUAAUAUAUUAUAUAAAUAUAUUAUAGAAGUAUAAUA...,.....(((((((.((((((((((((.(((((((((....)))))))...,EEEEESSSSSSSBSSSSSSSSSSSSBSSSSSSSSSHHHHSSSSSSS...,0.423,0,107,68,"[1.665, 2.1728, 2.0041, 1.2405, 0.620200000000...","[4.2139, 3.9637000000000002, 3.2467, 2.4716, 1...","[3.0942, 3.015, 2.1212, 2.0552, 0.881500000000...","[2.6717, 2.4818, 1.9919, 2.5484999999999998, 1...","[1.3285, 3.6173, 1.3057, 1.3021, 1.1507, 1.150...","[0.8267, 2.6577, 2.8481, 0.40090000000000003, ...","[2.1058, 3.138, 2.5437000000000003, 1.0932, 0....","[4.7366, 4.6243, 1.2068, 1.1538, 0.0, 0.0, 0.7...","[2.2052, 1.7947000000000002, 0.7457, 3.1233, 0...","[0.0, 5.1198, -0.3551, -0.3518, 0.0, 0.0, 0.0,..."


In [6]:
test.head(5)

,index,id,sequence,structure,predicted_loop_type,seq_length,seq_scored
0,0,id_00073f8be,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAA...,......((((((((((.(((((.....))))))))((((((((......,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHH...,107,68
1,1,id_000ae4237,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAU...,.....((((..((((((...(((((.....((((....)))).......,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIII...,130,91
2,2,id_00131c573,GGAAAACAAAACGGCCUGGAAGACGAAGGAAUUCGGCGCGAAGGCC...,...........((.(((.(.(..((..((..((((...))))..))...,EEEEEEEEEEESSISSSISISIISSIISSIISSSSHHHSSSSIISS...,107,68
3,3,id_00181fd34,GGAAAGGAUCUCUAUCGAAGGAUAGAGAUCGCUCGCGACGGCACGA...,......((((((((((....))))))))))((((((..((.(((.....,EEEEEESSSSSSSSSSHHHHSSSSSSSSSSSSSSSSIISSISSSHH...,107,68
4,4,id_0020473f7,GGAAACCCGCCCGCGCCCGCCCGCGCUGCUGCCGUGCCUCCUCUCC...,.....(((((((((((((((((((((((((((((((((((((((((...,EEEEESSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...,130,91


In [7]:
sub.head(5)

,id_seqpos,reactivity,deg_Mg_pH10,deg_pH10,deg_Mg_50C,deg_50C
0,id_00073f8be_0,0.0,0.0,0.0,0.0,0.0
1,id_00073f8be_1,0.0,0.0,0.0,0.0,0.0
2,id_00073f8be_2,0.0,0.0,0.0,0.0,0.0
3,id_00073f8be_3,0.0,0.0,0.0,0.0,0.0
4,id_00073f8be_4,0.0,0.0,0.0,0.0,0.0


In [8]:
train.describe()

,index,signal_to_noise,SN_filter,seq_length,seq_scored
count,2400.000000,2400.000000,2400.000000,2400.0,2400.0
mean,1199.500000,4.530456,0.662083,107.0,68.0
std,692.964646,2.835142,0.473099,0.0,0.0
min,0.000000,-0.103000,0.000000,107.0,68.0
25%,599.750000,2.391000,0.000000,107.0,68.0
50%,1199.500000,4.442500,1.000000,107.0,68.0
75%,1799.250000,6.294250,1.000000,107.0,68.0
max,2399.000000,17.194000,1.000000,107.0,68.0


In [9]:
test.describe()

,index,seq_length,seq_scored
count,3634.000000,3634.000000,3634.000000
mean,1816.500000,126.018987,87.018987
std,1049.189767,8.702624,8.702624
min,0.000000,107.000000,68.000000
25%,908.250000,130.000000,91.000000
50%,1816.500000,130.000000,91.000000
75%,2724.750000,130.000000,91.000000
max,3633.000000,130.000000,91.000000


In [17]:
repart_test1 = test.groupby('seq_scored').size()
repart_test1

seq_scored
68     629
91    3005
dtype: int64

In [18]:
repart_test2 = test.groupby('seq_length').size()
repart_test2

seq_length
107     629
130    3005
dtype: int64

In [15]:
train_verif = train.applymap(np.isreal).all(0)
train_verif

index                   True
id                     False
sequence               False
structure              False
predicted_loop_type    False
signal_to_noise         True
SN_filter               True
seq_length              True
seq_scored              True
dtype: bool

In [19]:
test_verif = train.applymap(np.isreal).all(0)
test_verif

index                   True
id                     False
sequence               False
structure              False
predicted_loop_type    False
signal_to_noise         True
SN_filter               True
seq_length              True
seq_scored              True
dtype: bool

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 19 columns):
index                  2400 non-null int64
id                     2400 non-null object
sequence               2400 non-null object
structure              2400 non-null object
predicted_loop_type    2400 non-null object
signal_to_noise        2400 non-null float64
SN_filter              2400 non-null int64
seq_length             2400 non-null int64
seq_scored             2400 non-null int64
reactivity_error       2400 non-null object
deg_error_Mg_pH10      2400 non-null object
deg_error_pH10         2400 non-null object
deg_error_Mg_50C       2400 non-null object
deg_error_50C          2400 non-null object
reactivity             2400 non-null object
deg_Mg_pH10            2400 non-null object
deg_pH10               2400 non-null object
deg_Mg_50C             2400 non-null object
deg_50C                2400 non-null object
dtypes: float64(1), int64(4), object(14)
memory usage: 356.4

## Analyse

In [41]:
sample = train.iloc[0]

In [44]:
sample['sequence']

'GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC'

In [48]:
dict(count(sample['sequence']))

{'G': 19, 'A': 45, 'C': 23, 'U': 20}

In [51]:
bases = []
for j in range(len(train)):
    counts = dict(count(train.iloc[j]['sequence']))
    bases.append((
        counts['A'] / 107,
        counts['G'] / 107,
        counts['C'] / 107,
        counts['U'] / 107
    ))
    
bases = pd.DataFrame(bases, columns=['A_percent', 'G_percent', 'C_percent', 'U_percent'])
bases

,A_percent,G_percent,C_percent,U_percent
0,0.420561,0.177570,0.214953,0.186916
1,0.233645,0.308411,0.299065,0.158879
2,0.401869,0.224299,0.186916,0.186916
3,0.261682,0.327103,0.327103,0.084112
4,0.542056,0.056075,0.056075,0.345794
...,...,...,...,...
2395,0.439252,0.205607,0.168224,0.186916
2396,0.672897,0.056075,0.196262,0.074766
2397,0.299065,0.252336,0.271028,0.177570
2398,0.448598,0.233645,0.158879,0.158879


In [49]:
dict(count(sample['structure']))

{'.': 61, '(': 23, ')': 23}

In [52]:
pairs_rate = []

for j in range(len(train)):
    res = dict(count(train.iloc[j]['structure']))
    pairs_rate.append(res['('] / 53.5)
    
pairs_rate = pd.DataFrame(pairs_rate, columns=['pairs_rate'])
pairs_rate

,pairs_rate
0,0.429907
1,0.616822
2,0.448598
3,0.598131
4,0.654206
...,...
2395,0.355140
2396,0.130841
2397,0.635514
2398,0.280374


In [53]:
pairs_dict = {}
queue = []
for i in range(0, len(sample['structure'])):
    if sample['structure'][i] == '(':
        queue.append(i)
    if sample['structure'][i] == ')':
        first = queue.pop()
        try:
            pairs_dict[(sample['sequence'][first], sample['sequence'][i])] += 1
        except:
            pairs_dict[(sample['sequence'][first], sample['sequence'][i])] = 1
pairs_dict

{('U', 'G'): 2, ('C', 'G'): 3, ('U', 'A'): 6, ('G', 'C'): 8, ('A', 'U'): 4}

In [ ]:
pairs = []
all_partners = []
for j in range(len(train)):
    partners = [-1 for i in range(130)]
    pairs_dict = {}
    queue = []
    for i in range(0, len(train.iloc[j]['structure'])):
        if train.iloc[j]['structure'][i] == '(':
            queue.append(i)
        if train.iloc[j]['structure'][i] == ')':
            first = queue.pop()
            try:
                pairs_dict[(train.iloc[j]['sequence'][first], train.iloc[j]['sequence'][i])] += 1
            except:
                pairs_dict[(train.iloc[j]['sequence'][first], train.iloc[j]['sequence'][i])] = 1
                
            partners[first] = i
            partners[i] = first
    
    all_partners.append(partners)
    
    pairs_num = 0
    pairs_unique = [('U', 'G'), ('C', 'G'), ('U', 'A'), ('G', 'C'), ('A', 'U'), ('G', 'U')]
    for item in pairs_dict:
        pairs_num += pairs_dict[item]
    add_tuple = list()
    for item in pairs_unique:
        try:
            add_tuple.append(pairs_dict[item]/pairs_num)
        except:
            add_tuple.append(0)
    pairs.append(add_tuple)
    
pairs = pd.DataFrame(pairs, columns=['U-G', 'C-G', 'U-A', 'G-C', 'A-U', 'G-U'])
pairs

In [ ]:
train['partners'] = all_partners

In [ ]:
pairs_dict = {}
queue = []
for j in range(len(train)):
    sam = train.iloc[j]
    for i in range(0, len(sam['structure'])):
        if sam['structure'][i] == '(':
            queue.append(i)
        if sam['structure'][i] == ')':
            first = queue.pop()
            try:
                pairs_dict[(sam['sequence'][first], sam['sequence'][i])] += 1
            except:
                pairs_dict[(sam['sequence'][first], sam['sequence'][i])] = 1
                
pairs_dict